In [0]:
print("krishnareddy")

krishnareddy


In [0]:
Read CSV file from file store

In [0]:
%fs
head dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/delimit_input-4.txt

Azar|BE|8|BigData|9273541255|Ramesh|Btech|3|pyspark|912354885|Krishna|MBA|2|sql|8106186796

In [0]:
df = spark.read.csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/delimit_input-4.txt")

In [0]:
df.show(truncate = False)
df.display()

+------------------------------------------------------------------------------------------+
|_c0                                                                                       |
+------------------------------------------------------------------------------------------+
|Azar|BE|8|BigData|9273541255|Ramesh|Btech|3|pyspark|912354885|Krishna|MBA|2|sql|8106186796|
+------------------------------------------------------------------------------------------+



_c0
Azar|BE|8|BigData|9273541255|Ramesh|Btech|3|pyspark|912354885|Krishna|MBA|2|sql|8106186796


In [0]:
#Apply regular expression Replace pipesymbol of every 5th occurance
#PySpark function explode(e: Column) is used to explode or create array or map columns to rows.
#When an array is passed to this function, it creates a new default column “col1” and it contains all array elements. When a map is passed, it creates two new columns one for key and one for value and each element in map split into the rows.
import pyspark
from pyspark.sql.functions import regexp_replace,split,explode
df1  = df.withColumn("check",regexp_replace("_c0","(.*?\\|){5}","$0-"))
df1.display()

_c0,check
Azar|BE|8|BigData|9273541255|Ramesh|Btech|3|pyspark|912354885|Krishna|MBA|2|sql|8106186796,Azar|BE|8|BigData|9273541255|-Ramesh|Btech|3|pyspark|912354885|-Krishna|MBA|2|sql|8106186796


In [0]:
df2=df1.withColumn("explode_colum",explode(split("check","\|-"))).select("explode_colum")

df2.display()

explode_colum
Azar|BE|8|BigData|9273541255
Ramesh|Btech|3|pyspark|912354885
Krishna|MBA|2|sql|8106186796


In [0]:
df2.select(split("explode_colum","\|")[3]).display()

"split(explode_colum, \|, -1)[3]"
BigData
pyspark
sql


In [0]:
#Convert DF to RDD and split by
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("convert DF to RDD").getOrCreate()
rddf2 = spark.sparkContext.parallelize([])
df2.select("explode_colum").rdd.map(lambda x:x[0].split("|")).collect()
rddf2 = df2.select("explode_colum").rdd.map(lambda x:x[0].split("|"))
df3=rddf2.toDF(["name","edu","yearofexp","tech","mobile"])
df3.display()



name,edu,yearofexp,tech,mobile
Azar,BE,8,BigData,9273541255
Ramesh,Btech,3,pyspark,912354885
Krishna,MBA,2,sql,8106186796


In [0]:
%fs
head dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/delimit_input___Copy.txt

ABCD|BE|8|BigData|9273541255|EFGH|Btech|3|pyspark|912354885|IJKL|MBA|2|sql|8106186796

In [0]:
df10 = spark.read.csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/delimit_input___Copy.txt")
df11=df10.withColumn("column",regexp_replace("_c0","(.*?\\|){5}","$0-"))
df22=df11.withColumn("exp_column",explode(split("column","\|-"))).select("exp_column")
df22.select(split("exp_column","\|")).display()



"split(exp_column, \|, -1)"
"List(ABCD, BE, 8, BigData, 9273541255)"
"List(EFGH, Btech, 3, pyspark, 912354885)"
"List(IJKL, MBA, 2, sql, 8106186796)"


In [0]:
spark1 = SparkSession.builder.appName("testing").getOrCreate()
#rdd11 = spark1.sparkContext.parallelize([])
rdd11=spark1.sparkContext.emptyRDD()
df22.select("exp_column").rdd.map(lambda x:x[0].split("|")).collect()
df22.display()
rdd11 = df22.select("exp_column").rdd.map(lambda x:x[0].split("|"))
df33 = rdd11.toDF(["name","edu","yearofexp","tech","mobile"])
df33.printSchema()
df33.display()


exp_column
ABCD|BE|8|BigData|9273541255
EFGH|Btech|3|pyspark|912354885
IJKL|MBA|2|sql|8106186796


root
 |-- name: string (nullable = true)
 |-- edu: string (nullable = true)
 |-- yearofexp: string (nullable = true)
 |-- tech: string (nullable = true)
 |-- mobile: string (nullable = true)



name,edu,yearofexp,tech,mobile
ABCD,BE,8,BigData,9273541255
EFGH,Btech,3,pyspark,912354885
IJKL,MBA,2,sql,8106186796


In [0]:
Q2: consider a file with corrupt/bad data as show in the file how will you load and handle the bad data into spark dataframe?
dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_2.txt

spark.read.option("mode","FAILFAST").csv()

modes are 3 types 
1. PERMISSIVE --> DEFAULT MODE
2.FAILFAST -->stop processing the bad file further.
3. DROPMALFORMED -->remove the bad data and process the file



In [0]:
%fs
head
dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_2.txt

emp_no,emp_name,department
101,reddy,healthcare
invalid entry,decription:bad record entry
102,Krishna,IT
connection lost,description:poor connection
104,pavan,HR
Bad Record,description:corrupt record

In [0]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("baddata").getOrCreate()
rdd = spark.sparkContext.parallelize([])
df99 = spark.read.option("mode","FAILFAST").csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_2.txt",header = True)
df99.display()

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7) (ip-10-172-205-67.us-west-2.compute.internal executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:REDACTED_LOCAL_PART@gmail.com/Question_2.txt.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:521)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:494)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$hasNext$1(FileScanRDD.scala:356)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:351)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:80)
	at org.apache.spark.sql.execution.collect.Collector.$anonfun$processFunc$1(Collector.scala:155)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:156)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:125)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:95)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$13(Executor.scala:832)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1681)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:835)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:690)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1384)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:103)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:458)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at org.apache.spark.util.CompletionIterator.hasNext(CompletionIterator.scala:31)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:459)
	... 26 more
Caused by: org.apache.spark.sql.catalyst.util.BadRecordException: org.apache.spark.sql.catalyst.csv.MalformedCSVException: Malformed CSV record
Caused by: org.apache.spark.sql.catalyst.csv.MalformedCSVException: Malformed CSV record

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:3029)
	at org.apache.spark.scheduler.DAGS

In [0]:
df98 = spark.read.option("mode","DROPMALFORMED").csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_2.txt",header = True)
df98.display()
df97 = spark.read.option("mode","PERMISSIVE").csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_2.txt",header = True)
df97.display()

emp_no,emp_name,department
101,reddy,healthcare
102,Krishna,IT
104,pavan,HR


emp_no,emp_name,department
101,reddy,healthcare
invalid entry,decription:bad record entry,null
102,Krishna,IT
connection lost,description:poor connection,null
104,pavan,HR
Bad Record,description:corrupt record,null


In [0]:
Q3:consider a file with delimted ~| how will you load it in the spark data frame?


In [0]:
%fs
head
dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_3.txt

Name~|age
Krishna,Reddy~|38
Gali,Mani~|26
MR,Esuru~|25

In [0]:
#first method
df33 = spark.read.option("mode","permissive").option("delimiter","~|").csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_3.txt",header = True)
df33.display()

Name,age
"Krishna,Reddy",38
"Gali,Mani",26
"MR,Esuru",25


In [0]:
#second method
df44= spark.read.text("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_3.txt")
header = df44.first()[0]
header
schema = header.split("~|")
schema
#df44.filter(df44["value"] != header).show()
df44.filter(df44["value"] != header).rdd.map(lambda x:x[0].split("~|")).collect()
df55= df44.filter(df44["value"] != header).rdd.map(lambda x:x[0].split("~|"))
df66 = df55.toDF(schema)
df66.show(truncate = False)
#df44.show(truncate = False)

+-----------------+
|            value|
+-----------------+
|Krishna,Reddy~|38|
|    Gali,Mani~|26|
|     MR,Esuru~|25|
+-----------------+

+-------------+---+
|Name         |age|
+-------------+---+
|Krishna,Reddy|38 |
|Gali,Mani    |26 |
|MR,Esuru     |25 |
+-------------+---+



In [0]:
Q4:merge two dataframe in pyspark
dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_4.txt
dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_4A.txt

In [0]:
%fs
head
dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_4.txt

Name|Age
Krishna,Reddt|35
mani,gali|26

In [0]:
%fs
head
dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_4A.txt

Name|Age|Gender
Esuru,Kurmarao|32|M
mohini,gali|26|F
Potti,mani|25|F

In [0]:
df33 = spark.read.option("delimiter","|").csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_4.txt",header = True)
df33.display()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-3001292341343909> in <module>
----> 1 df33 = spark.read.option("delimiter","|").csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_4.txt",header = True,schema=schema)
      2 df33.display()

NameError: name 'schema' is not defined

In [0]:
df31 = spark.read.option("delimiter","|").csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_4A.txt", header = True)
df31.display()

Name,Age,Gender
"Esuru,Kurmarao",32,M
"mohini,gali",26,F
"Potti,mani",25,F


In [0]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
#df33.union(df31)
#First method withcolumn and union
df34 =df33.withColumn("Gender",lit("null"))
df34.show()
df34.union(df31).show()


+-------------+---+------+
|         Name|Age|Gender|
+-------------+---+------+
|Krishna,Reddt| 35|  null|
|    mani,gali| 26|  null|
+-------------+---+------+

+--------------+---+------+
|          Name|Age|Gender|
+--------------+---+------+
| Krishna,Reddt| 35|  null|
|     mani,gali| 26|  null|
|Esuru,Kurmarao| 32|     M|
|   mohini,gali| 26|     F|
|    Potti,mani| 25|     F|
+--------------+---+------+



In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
#second method Schema and union
schema = StructType([
            StructField("name",StringType(),True),
            StructField("age",IntegerType(),True),
            StructField("gender",StringType(),True)
])

schema1 = StructType([
            StructField("name",StringType(),True),
            StructField("age",IntegerType(),True),
            StructField("gender",StringType(),True)
])

df44 = spark.read.option("delimiter","|").csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_4.txt",header = True,schema=schema)
df44.display()
df55 = spark.read.option("delimiter","|").csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_4A.txt", header = True,schema=schema1)
df55.display()
df44.union(df55).show()

name,age,gender
"Krishna,Reddt",35,null
"mani,gali",26,null


Name,Age,Gender
"Esuru,Kurmarao",32,M
"mohini,gali",26,F
"Potti,mani",25,F


+--------------+---+------+
|          name|age|gender|
+--------------+---+------+
| Krishna,Reddt| 35|  null|
|     mani,gali| 26|  null|
|Esuru,Kurmarao| 32|     M|
|   mohini,gali| 26|     F|
|    Potti,mani| 25|     F|
+--------------+---+------+



In [0]:
#third method outer join
df66=df44.join(df55,df44.name==df55.name,"outer")
df66=df44.join(df55,on=["name","age","gender"],how ="outer")
df66.show(truncate = False)

+--------------+---+------+
|name          |age|gender|
+--------------+---+------+
|Krishna,Reddt |35 |null  |
|mani,gali     |26 |null  |
|Esuru,Kurmarao|32 |M     |
|Potti,mani    |25 |F     |
|mohini,gali   |26 |F     |
+--------------+---+------+



In [0]:
from pyspark.sql.functions import *
#fourth step automated approach
df33.columns
df31.columns
df66 = spark.read.option("delimiter","|").csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_4.txt",header = True)

df77 = spark.read.option("delimiter","|").csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_4A.txt", header = True)


listA = list(set(df66.columns)-set(df77.columns))
listB = list(set(df77.columns)-set(df66.columns))
for i in listA:
    df77=df77.withColumn(i,lit("null"))
    
for i in listB:
    df66=df66.withColumn(i,lit("null"))
    
df77.union(df66).show()

+--------------+---+------+
|          Name|Age|Gender|
+--------------+---+------+
|Esuru,Kurmarao| 32|     M|
|   mohini,gali| 26|     F|
|    Potti,mani| 25|     F|
| Krishna,Reddt| 35|  null|
|     mani,gali| 26|  null|
+--------------+---+------+



In [0]:
calculate expiry data by remainig date and expiry date
dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_6.txt

In [0]:
%fs
head
dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_6.txt

rechargeid|rechargedate|remainingdays|validity
R201623|20200511|1|online
R201624|20200119|110|online
R201625|20200105|35|online
R201626|20191105|215|online

In [0]:
import pyspark
from pyspark.sql.types import StringType
from pyspark.sql.functions import *
df66 = spark.read.option("mode","PERMISSIVE")\
                .option("delimiter","|")\
                .option("inferschema",True)\
                .csv("dbfs:/FileStore/shared_uploads/krishnareddyesuru@gmail.com/Question_6.txt",header=True)
#df66.display()
#df66.printSchema()
#df66.withColumn("expiry_date",date_add(to_date(col("rechargedate"),"yyyy-mm-dd"),"remainingdays"))
df61=df66.withColumn("rechargedate",to_date(col("rechargedate").cast(StringType()),"yyyyMMdd"))
df61.show()
df61.printSchema()
#df61=df66.select(col("rechargedate").cast(StringType()).alias("to_string"))
#df61=df66.withColumn("rechargedate1",col("rechargedate").cast(StringType()))
#df62=df61.withColumn("rechargedate",to_date(col("rechargedate1"),"yyyyMMdd"))
#df62.show()
#df62.printSchema()
#df66.show()
#df66.printSchema()

+----------+------------+-------------+--------+
|rechargeid|rechargedate|remainingdays|validity|
+----------+------------+-------------+--------+
|   R201623|  2020-05-11|            1|  online|
|   R201624|  2020-01-19|          110|  online|
|   R201625|  2020-01-05|           35|  online|
|   R201626|  2019-11-05|          215|  online|
+----------+------------+-------------+--------+

root
 |-- rechargeid: string (nullable = true)
 |-- rechargedate: date (nullable = true)
 |-- remainingdays: integer (nullable = true)
 |-- validity: string (nullable = true)



In [0]:
Q10:Convert RDD to Dataframe & Dataframe to RDD

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark=SparkSession.builder.master("local").appName("example").getOrCreate()
data = [("krishna","reddy","esuru")]
rdd = spark.sparkContext.parallelize(data)
#rdd1=spark.SparkContext.emptyRDD([])
rdd.collect()

#type(rdd)
df=rdd.toDF(["first","middle","last"])
#type(df)
df.show()

+-------+------+-----+
|  first|middle| last|
+-------+------+-----+
|krishna| reddy|esuru|
+-------+------+-----+

